In [27]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('users_behavior.csv')


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
features = df[['calls', 'minutes', 'messages', 'mb_used']] # extrae las características
target = df['is_ultra'] # extrae los objetivos

features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, random_state=12345)
model = LinearRegression()
model.fit(features_train, target_train)
pred_valid= model.predict(features_valid)

In [36]:
resul = mean_squared_error(target_valid, pred_valid)**0.5
resul

np.float64(0.43786312404942046)

In [32]:
best_error = 10000 # configura el inicio de RECM
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model =RandomForestRegressor(random_state=54321, n_estimators=est, max_depth=depth)
        model.fit(features_train,target_train) # entrena el modelo en el conjunto de entrenamiento
        predictions_valid = model.predict(features_valid) # obtén las predicciones del modelo en el conjunto de validación
        error = mean_squared_error(target_valid, predictions_valid)**0.5 # calcula la RECM en el conjunto de validación
        print("Validación RECM para los n_estimators de", est, ", depth=", depth, "is", error)
        if error < best_error: # guardamos la configuración del modelo si se logra el error más bajo
            best_error = error
            best_est = est
            best_depth = depth

print("RECM del mejor modelo en el conjunto de validación:", best_error, "n_estimators:", best_est, "best_depth:", best_depth)

Validación RECM para los n_estimators de 10 , depth= 1 is 0.4246062261876908
Validación RECM para los n_estimators de 10 , depth= 2 is 0.4073844843171148
Validación RECM para los n_estimators de 10 , depth= 3 is 0.4028235874867796
Validación RECM para los n_estimators de 10 , depth= 4 is 0.3946600922547636
Validación RECM para los n_estimators de 10 , depth= 5 is 0.39311339225605324
Validación RECM para los n_estimators de 10 , depth= 6 is 0.3931432440598083
Validación RECM para los n_estimators de 10 , depth= 7 is 0.39125163171342353
Validación RECM para los n_estimators de 10 , depth= 8 is 0.3905873202135934
Validación RECM para los n_estimators de 10 , depth= 9 is 0.3891370997177105
Validación RECM para los n_estimators de 10 , depth= 10 is 0.3919901399844317
Validación RECM para los n_estimators de 20 , depth= 1 is 0.42586879089000107
Validación RECM para los n_estimators de 20 , depth= 2 is 0.40631734000380504
Validación RECM para los n_estimators de 20 , depth= 3 is 0.40099217713